In [1]:
!hdfs dfsadmin -report

Configured Capacity: 302585180160 (281.80 GB)
Present Capacity: 287068499968 (267.35 GB)
DFS Remaining: 287067676672 (267.35 GB)
DFS Used: 823296 (804 KB)
DFS Used%: 0.00%
Replicated Blocks:
	Under replicated blocks: 0
	Blocks with corrupt replicas: 0
	Missing blocks: 0
	Missing blocks (with replication factor 1): 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0
Erasure Coded Block Groups: 
	Low redundancy block groups: 0
	Block groups with corrupt internal blocks: 0
	Missing block groups: 0
	Low redundancy blocks with highest priority to recover: 0
	Pending deletion blocks: 0

-------------------------------------------------
Live datanodes (3):

Name: 10.10.1.1:9866 (node0-link-1)
Hostname: c220g5-111221vm-1.wisc.cloudlab.us
Decommission Status : Normal
Configured Capacity: 100861726720 (93.93 GB)
DFS Used: 274432 (268 KB)
Non DFS Used: 4370432 (4.17 MB)
DFS Remaining: 95686344704 (89.11 GB)
DFS Used%: 0.00%
DFS Remaining%: 94.87%
Configured Ca

In [2]:
import requests

master_url = 'http://10.10.1.1:8080/json/'

# Fetch the details
response = requests.get(master_url)
data = response.json()

# Print out the data
print(data)


{'url': 'spark://10.10.1.1:7077', 'workers': [{'id': 'worker-20240203000733-10.10.1.3-35519', 'host': '10.10.1.3', 'port': 35519, 'webuiaddress': 'http://10.10.1.3:8081', 'cores': 5, 'coresused': 0, 'coresfree': 5, 'memory': 31079, 'memoryused': 0, 'memoryfree': 31079, 'resources': {}, 'resourcesused': {}, 'resourcesfree': {}, 'state': 'ALIVE', 'lastheartbeat': 1706993239327}, {'id': 'worker-20240203000732-10.10.1.2-36699', 'host': '10.10.1.2', 'port': 36699, 'webuiaddress': 'http://10.10.1.2:8081', 'cores': 5, 'coresused': 0, 'coresfree': 5, 'memory': 31079, 'memoryused': 0, 'memoryfree': 31079, 'resources': {}, 'resourcesused': {}, 'resourcesfree': {}, 'state': 'ALIVE', 'lastheartbeat': 1706993239268}, {'id': 'worker-20240203000733-10.10.1.1-46595', 'host': '10.10.1.1', 'port': 46595, 'webuiaddress': 'http://10.10.1.1:8081', 'cores': 5, 'coresused': 0, 'coresfree': 5, 'memory': 31079, 'memoryused': 0, 'memoryfree': 31079, 'resources': {}, 'resourcesused': {}, 'resourcesfree': {}, 'st

In [3]:
!wget -O /users/lyang439/export.csv http://pages.cs.wisc.edu/~shivaram/cs744-fa18/assets/export.csv

--2024-02-03 14:47:33--  http://pages.cs.wisc.edu/~shivaram/cs744-fa18/assets/export.csv
Resolving pages.cs.wisc.edu (pages.cs.wisc.edu)... 128.105.7.9
Connecting to pages.cs.wisc.edu (pages.cs.wisc.edu)|128.105.7.9|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://pages.cs.wisc.edu/~shivaram/cs744-fa18/assets/export.csv [following]
--2024-02-03 14:47:33--  https://pages.cs.wisc.edu/~shivaram/cs744-fa18/assets/export.csv
Connecting to pages.cs.wisc.edu (pages.cs.wisc.edu)|128.105.7.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 109949 (107K) [text/csv]
Saving to: ‘/users/lyang439/export.csv’

/users/lyang439/exp 100%[===================>] 107.37K  --.-KB/s    in 0.004s  

2024-02-03 14:47:33 (24.4 MB/s) - ‘/users/lyang439/export.csv’ saved [109949/109949]



In [4]:
!hdfs dfs -mkdir /data

mkdir: `/data': File exists


In [5]:
!hdfs dfs -ls /

Found 1 items
drwxr-xr-x   - lyang439 supergroup          0 2024-02-03 00:44 /data


In [8]:
!hdfs dfs -put ~/export.csv /data

In [9]:
!hdfs dfs -ls /data

Found 1 items
-rw-r--r--   3 lyang439 supergroup     109949 2024-02-03 00:44 /data/export.csv


In [6]:
from pyspark.sql import SparkSession

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("Sort IoT Data") \
    .getOrCreate()

# Load the data
df = spark.read.csv('hdfs://10.10.1.1:9000/data/export.csv', inferSchema=True, header=True)

# Sort the data
sorted_df = df.sort(['cca2', 'timestamp'])

# Show or save the results
#sorted_df.show()
sorted_df.write.csv('hdfs://10.10.1.1:9000/data/export_sorted.csv', mode='overwrite', header=True)

spark.stop()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/03 13:48:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/users/lyang439/.local/lib/python3.7/site-packages/pyspark/context.py:317: FutureWarning: Python 3.7 support is deprecated in Spark 3.4.
  warnings.warn("Python 3.7 support is deprecated in Spark 3.4.", FutureWarning)


In [ ]:
!hdfs dfs -cat /data/export_sorted.csv/*